<a href="https://colab.research.google.com/github/SRIRAAGAVI/word-dbms/blob/main/THE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/andrewcorum/steganalysis?select=landscape-pictures")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: sriraagavi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewcorum/steganalysis


100%|██████████| 66.6M/66.6M [00:00<00:00, 101MB/s]


LSB

In [26]:
import cv2
import os
import numpy as np

def text_to_binary(text):
    """Convert text to a binary string."""
    return ''.join(format(ord(char), '08b') for char in text)

def embed_lsb(image_path, binary_payload, output_path):
    """
    Embed a binary payload into an image using LSB steganography.
    Args:
    - image_path: Path to the input image.
    - binary_payload: Binary string of the payload to embed.
    - output_path: Path to save the output image.
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Unable to load image: {image_path}")

    # Get image dimensions
    height, width, channels = image.shape
    max_capacity = height * width * 3  # RGB has 3 channels per pixel

    # Check payload size
    if len(binary_payload) > max_capacity:
        raise ValueError("Payload too large for the image capacity.")

    # Flatten the image array to iterate through all pixel values
    flat_image = image.flatten()

    # Embed the payload
    for i in range(len(binary_payload)):
        flat_image[i] = (flat_image[i] & 0xFE) | int(binary_payload[i])  # Modify only the LSB

    # Reshape the modified array back to the original image shape
    embedded_image = flat_image.reshape(height, width, channels)

    # Save the stego image
    cv2.imwrite(output_path, embedded_image)
    print(f"Payload embedded successfully: {output_path}")

def embed_payload_to_images(input_folder, payload, output_folder, num_images):
    """
    Embed the payload into selected number of images in the folder using LSB.
    Args:
    - input_folder: Folder containing original images.
    - payload: Text payload to embed.
    - output_folder: Folder to save stego images.
    - num_images: Number of images to process.
    """
    # Convert the payload to binary
    binary_payload = text_to_binary(payload)

    # Get list of images
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    image_files.sort()

    # Create output folder if not exists
    os.makedirs(output_folder, exist_ok=True)

    # Process selected images
    for i in range(min(num_images, len(image_files))):
        input_path = os.path.join(input_folder, image_files[i])
        output_path = os.path.join(output_folder, f"stego_{image_files[i]}")

        # Embed payload into each image
        embed_lsb(input_path, binary_payload, output_path)

# Example usage
input_folder = "/content/steganalysis/landscape-pictures/cover"  # Folder with original images
output_folder = "/content/steganalysis/landscape-pictures/output-LSB"  # Folder to save stego images
payload = "This is a secret message"  # Payload to embed
num_images = 20  # Number of images to process

# Embed payload into images
embed_payload_to_images(input_folder, payload, output_folder, num_images)


Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000.jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(2).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(3).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(4).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(5).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(6).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000000_(7).jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000001.jpg
Payload embedded successfully: /content/steganalysis/landscape-pictures/output-LSB/stego_00000001_(2).jpg
Payload embedded successfully: /content/steganalysis/l

Comparison of the original nd ebedded img = PSNR


In [27]:
import cv2
import os
import numpy as np

def calculate_psnr(original, stego):
    """Calculate the PSNR between two images."""
    # Convert images to float32 for precise calculations
    original = original.astype(np.float32)
    stego = stego.astype(np.float32)

    mse = np.mean((original - stego) ** 2)
    if mse == 0:
        return 100  # Infinite PSNR (practically a high value like 100)
    max_pixel = 255.0  # Max pixel value for 8-bit images
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_psnr_for_images(original_folder, stego_folder, num_images):
    """Calculate PSNR for a selected number of images from two folders."""
    # Get the list of image files in both folders
    original_files = [f for f in os.listdir(original_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    stego_files = [f for f in os.listdir(stego_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Sort the files to ensure matching pairs (by name or order)
    original_files.sort()
    stego_files.sort()

    # Process the selected number of images
    for i in range(min(num_images, len(original_files), len(stego_files))):
        original_path = os.path.join(original_folder, original_files[i])
        stego_path = os.path.join(stego_folder, stego_files[i])

        # Load the images
        original_image = cv2.imread(original_path, cv2.IMREAD_COLOR)
        stego_image = cv2.imread(stego_path, cv2.IMREAD_COLOR)

        # Check if images were loaded correctly
        if original_image is None:
            print(f"Error: Unable to load original image {original_files[i]}!")
            continue
        if stego_image is None:
            print(f"Error: Unable to load stego image {stego_files[i]}!")
            continue

        # Ensure both images are of the same size
        if original_image.shape != stego_image.shape:
            print(f"Error: Images {original_files[i]} and {stego_files[i]} must have the same dimensions!")
            continue

        # Calculate PSNR
        psnr_value = calculate_psnr(original_image, stego_image)
        print(f"PSNR for {original_files[i]} and {stego_files[i]}: {psnr_value:.2f} dB")

# Example usage
original_folder = "/content/steganalysis/landscape-pictures/cover"
stego_folder = "/content/steganalysis/landscape-pictures/output-LSB"
num_images = 20  # Number of image pairs to process

# Calculate PSNR for the images
calculate_psnr_for_images(original_folder, stego_folder, num_images)


PSNR for 00000000.jpg and stego_00000000.jpg: 46.38 dB
PSNR for 00000000_(2).jpg and stego_00000000_(2).jpg: 42.28 dB
PSNR for 00000000_(3).jpg and stego_00000000_(3).jpg: 34.96 dB
PSNR for 00000000_(4).jpg and stego_00000000_(4).jpg: 43.08 dB
PSNR for 00000000_(5).jpg and stego_00000000_(5).jpg: 40.06 dB
PSNR for 00000000_(6).jpg and stego_00000000_(6).jpg: 37.21 dB
PSNR for 00000000_(7).jpg and stego_00000000_(7).jpg: 46.82 dB
PSNR for 00000001.jpg and stego_00000001.jpg: 36.78 dB
PSNR for 00000001_(2).jpg and stego_00000001_(2).jpg: 38.37 dB
PSNR for 00000001_(3).jpg and stego_00000001_(3).jpg: 38.41 dB
PSNR for 00000001_(4).jpg and stego_00000001_(4).jpg: 43.96 dB
PSNR for 00000001_(5).jpg and stego_00000001_(5).jpg: 34.82 dB
PSNR for 00000001_(6).jpg and stego_00000001_(6).jpg: 40.84 dB
PSNR for 00000001_(7).jpg and stego_00000001_(7).jpg: 45.98 dB
PSNR for 00000002.jpg and stego_00000002.jpg: 30.80 dB
PSNR for 00000002_(2).jpg and stego_00000002_(2).jpg: 33.93 dB
PSNR for 0000000

In [8]:
import cv2
import os
import numpy as np

def embed_payload_into_image(image, payload, block_size=8):
    # Ensure image is a multiple of the block size (if needed, pad the image)
    h, w = image.shape
    if h % block_size != 0 or w % block_size != 0:
        h_new = (h // block_size) * block_size
        w_new = (w // block_size) * block_size
        image = image[:h_new, :w_new]

    # Convert the payload into a binary array
    binary_payload = ''.join(format(ord(c), '08b') for c in payload)

    # Create an empty list to store the DCT blocks
    dct_blocks = []
    payload_index = 0

    # Iterate through the image in blocks of 8x8
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            block = image[i:i + block_size, j:j + block_size]

            # Apply DCT to the block
            dct_block = cv2.dct(np.float32(block))

            # Embed the payload in the DCT coefficients (low-frequency coefficients)
            for x in range(block_size):
                for y in range(block_size):
                    if payload_index < len(binary_payload):
                        # Modify the LSB of a DCT coefficient to embed the payload bit
                        if binary_payload[payload_index] == '1':
                            dct_block[x, y] = dct_block[x, y] + 1
                        else:
                            dct_block[x, y] = dct_block[x, y] - 1
                        payload_index += 1

            # Add the modified block to the list
            dct_blocks.append(dct_block)

    # Reconstruct the image from the modified DCT blocks
    new_image = np.zeros_like(image, dtype=np.float32)
    block_idx = 0
    for i in range(0, h, block_size):
        for j in range(0, w, block_size):
            new_image[i:i + block_size, j:j + block_size] = cv2.idct(dct_blocks[block_idx])
            block_idx += 1

    # Normalize the new image to uint8 format
    new_image = np.uint8(np.clip(new_image, 0, 255))

    return new_image

def embed_payload_in_images(input_folder, output_folder, payload, num_images, block_size=8):
    # Create the output folder if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Process only the selected number of images
    for idx, image_file in enumerate(image_files[:num_images]):
        image_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, f"stego_{image_file}")

        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Skipping {image_file}: Unable to read the image.")
            continue

        # Embed the payload
        stego_image = embed_payload_into_image(image, payload, block_size)

        # Save the stego image
        cv2.imwrite(output_path, stego_image)
        print(f"Payload embedded into {image_file} and saved as {output_path}")

# Example usage
input_folder = "/content/steganalysis/landscape-pictures/cover"
output_folder = "/content/steganalysis/landscape-pictures/output"
payload = "This is a very secret msg which shoul be very confidentialto anyone"
num_images = 20  # Number of images to process

# Embed the payload into the images
embed_payload_in_images(input_folder, output_folder, payload, num_images)


Payload embedded into 00000351_(2).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000351_(2).jpg
Payload embedded into 00000355_(3).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000355_(3).jpg
Payload embedded into 00000506_(5).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000506_(5).jpg
Payload embedded into 00000083_(3).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000083_(3).jpg
Payload embedded into 00000143_(5).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000143_(5).jpg
Payload embedded into 00000114_(3).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000114_(3).jpg
Payload embedded into 00000370_(3).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000370_(3).jpg
Payload embedded into 00000426_(2).jpg and saved as /content/steganalysis/landscape-pictures/output/stego_00000426_(2).jpg
Payload embedded

comparison of the two dct imgs original and embedded =PSNR

In [10]:
import cv2
import os
import numpy as np

def calculate_psnr(original, stego):
    """Calculate the PSNR between two images."""
    mse = np.mean((original - stego) ** 2)
    if mse == 0:
        return 100  # If images are identical, PSNR is infinite (returning 100 as a high value)
    max_pixel = 255.0  # For 8-bit images
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_psnr_for_images(original_folder, stego_folder, num_images):
    """Calculate PSNR for a selected number of images from two folders."""
    # Get the list of image files in both folders
    original_files = [f for f in os.listdir(original_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    stego_files = [f for f in os.listdir(stego_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Sort the files to ensure matching pairs (by name or order)
    original_files.sort()
    stego_files.sort()

    # Process the selected number of images
    for i in range(min(num_images, len(original_files), len(stego_files))):
        original_path = os.path.join(original_folder, original_files[i])
        stego_path = os.path.join(stego_folder, stego_files[i])

        # Load the images
        original_image = cv2.imread(original_path)
        stego_image = cv2.imread(stego_path)

        # Check if images were loaded correctly
        if original_image is None:
            print(f"Error: Unable to load original image {original_files[i]}!")
            continue
        if stego_image is None:
            print(f"Error: Unable to load stego image {stego_files[i]}!")
            continue

        # Ensure both images are of the same size
        if original_image.shape != stego_image.shape:
            print(f"Error: Images {original_files[i]} and {stego_files[i]} must have the same dimensions!")
            continue

        # Calculate PSNR
        psnr_value = calculate_psnr(original_image, stego_image)
        print(f"PSNR for {original_files[i]} and {stego_files[i]}: {psnr_value:.2f} dB")

# Example usage
original_folder = "/content/steganalysis/landscape-pictures/cover"
stego_folder = "/content/steganalysis/landscape-pictures/output-DCT"
num_images = 20  # Number of image pairs to process

# Calculate PSNR for the images
calculate_psnr_for_images(original_folder, stego_folder, num_images)


PSNR for 00000000.jpg and stego_00000001_(7).jpg: 27.91 dB
PSNR for 00000000_(2).jpg and stego_00000070_(7).jpg: 27.87 dB
PSNR for 00000000_(3).jpg and stego_00000083_(3).jpg: 27.99 dB
PSNR for 00000000_(4).jpg and stego_00000114_(3).jpg: 27.96 dB
PSNR for 00000000_(5).jpg and stego_00000143_(5).jpg: 27.87 dB
PSNR for 00000000_(6).jpg and stego_00000238_(5).jpg: 27.94 dB
PSNR for 00000000_(7).jpg and stego_00000251_(4).jpg: 27.90 dB
PSNR for 00000001.jpg and stego_00000253.jpg: 28.01 dB
PSNR for 00000001_(2).jpg and stego_00000351_(2).jpg: 27.81 dB
PSNR for 00000001_(3).jpg and stego_00000355_(3).jpg: 27.79 dB
PSNR for 00000001_(4).jpg and stego_00000370_(3).jpg: 27.92 dB
PSNR for 00000001_(5).jpg and stego_00000390_(5).jpg: 27.94 dB
PSNR for 00000001_(6).jpg and stego_00000396_(2).jpg: 27.90 dB
PSNR for 00000001_(7).jpg and stego_00000426_(2).jpg: 27.81 dB
PSNR for 00000002.jpg and stego_00000458_(4).jpg: 27.93 dB
PSNR for 00000002_(2).jpg and stego_00000506_(5).jpg: 27.91 dB
PSNR for

In [25]:
import shutil

shutil.rmtree('/content/steganalysis/landscape-pictures/output-LSB')